# Averaging Function

In [1]:
from pathlib import Path
import sys
sys.path.append('../')
from pma_open import *


In [2]:
file_path = '../hel1.pma'
avg_frame_png(file_path)

image_path = "hel1_Avg_Frame/hel1_Avg_Frame.png"

Image Size: 512 x 512
Image Size: 512 x 512
Sucessfully generated average frame
Average frame saved as: hel1_Avg_Frame.png


In [3]:
avg_frame_data = avg_frame_arr(file_path)
print(f"Average Frame Data- shape: {avg_frame_data.shape}")
print(f"Average Frame Data- Type: {type(avg_frame_data)}")
print(f"Average Frame Data- dType: {avg_frame_data.dtype}")
print(f"First line of Average Frame Data: {avg_frame_data[0]}")

Image Size: 512 x 512
Sucessfully generated average frame
Average Frame Data- shape: (512, 512)
Average Frame Data- Type: <class 'numpy.ndarray'>
Average Frame Data- dType: uint8
First line of Average Frame Data: [ 7  7  7  7  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9 10 10 10 10 10 10 10 10 10 11 11 11 11 11 12 12 12 12 13 14
 15 16 17 17 17 17 17 17 18 22 31 40 42 34 24 19 17 16 14 13 13 12 12 13
 14 14 13 13 13 13 12 12 12 12 11 11 11 11 12 12 12 12 13 13 13 19 35 58
 70 60 35 18 15 15 14 12 12 12 12 12 12 12 11 11 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10  9  9 10 10 10 10 10  9  9  9  9  9 10 10  9  9 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 12 12 12 11 12 15
 20 25 26 21 15 13 12 12 11 11 11 11 11 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10  9  9  9  9  9  9  9  9  9  9
 10 10 10 11 10 10 10  9  9  9  9  9  9  9  9  9  9  9  9

In [4]:
avg_frame_path= "/hel1_Avg_Frame/hel1_Avg_Frame.png"
src = cv2.imread(avg_frame_path)
gray_img = cv2.cvtColor(avg_frame_path, cv2.COLOR_BGR2GRAY) 
  
# # Shi-Tomasi corner detection function 
# # We are detecting only 100 best corners here 
# # You can change the number to get desired result. 
# corners = cv2.goodFeaturesToTrack(gray_img, 100, 0.01, 10) 
  
# # convert corners values to integer 
# # So that we will be able to draw circles on them 
# corners = np.int0(corners) 
  
# # draw red color circles on all corners 
# for i in corners: 
#     x, y = i.ravel() 
#     cv2.circle(img, (x, y), 3, (255, 0, 0), -1) 
  
# # resulting image 
# plt.imshow(img) 
  
# # De-allocate any associated memory usage   
# if cv2.waitKey(0) & 0xff == 27:  
#     cv2.destroyAllWindows() 

[ WARN:0@2.119] global loadsave.cpp:268 findDecoder imread_('/hel1_Avg_Frame/hel1_Avg_Frame.png'): can't open/read file: check file path/integrity


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
# corners = cv2.goodFeaturesToTrack(avg_frame_data,25,0.01,10)
# corners = np.int0(corners)

# for i in corners:
#     x,y = i.ravel()
#     cv2.circle(img,(x,y),3,255,-1)
 
# plt.imshow(img),plt.show()

error: OpenCV(4.11.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/corner.cpp:254: error: (-215:Assertion failed) src.type() == CV_8UC1 || src.type() == CV_32FC1 in function 'cornerEigenValsVecs'


In [ ]:
# convert image to grayscale 

gray_img = cv2.cvtColor(avg_frame_data, cv2.COLOR_BGR2GRAY) 
  
# # Shi-Tomasi corner detection function 
# # We are detecting only 100 best corners here 
# # You can change the number to get desired result. 
# corners = cv2.goodFeaturesToTrack(gray_img, 100, 0.01, 10) 
  
# # convert corners values to integer 
# # So that we will be able to draw circles on them 
# corners = np.int0(corners) 
  
# # draw red color circles on all corners 
# for i in corners: 
#     x, y = i.ravel() 
#     cv2.circle(img, (x, y), 3, (255, 0, 0), -1) 
  
# # resulting image 
# plt.imshow(img) 
  
# # De-allocate any associated memory usage   
# if cv2.waitKey(0) & 0xff == 27:  
#     cv2.destroyAllWindows() 

error: OpenCV(4.11.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-15:Bad number of channels) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4>, cv::impl::(anonymous namespace)::Set<1>, cv::impl::(anonymous namespace)::Set<0, 2, 5>>::CvtHelper(InputArray, OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4>, VDcn = cv::impl::(anonymous namespace)::Set<1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 2, 5>, sizePolicy = cv::impl::(anonymous namespace)::NONE]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
